# Lab 4: Develop a Multi-Agent System

In this lab, we build a modern, production-ready multi-agent system using the latest Azure AI Python SDKs and best practices.
- Each agent is created as a connected agent using Azure AI Agent Service.
- Orchestration is performed using direct agent-to-agent calls, not just a group chat or plugin pattern.
- Uses official Microsoft documentation patterns for agent creation, tool/resource registration, and message passing.


### Part 1: Create the Search, Report, and Validation Agents

#### Step 1: Load packages

In [1]:
import os
import json
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    AzureAISearchToolDefinition, AzureAISearchToolResource, AISearchIndexResource, ToolResources, AgentThreadCreationOptions
)

# Load environment variables
load_dotenv()

#### Step 2: Connect to your Microsoft Foundry Project

In [ ]:
# Connecting to our Microsoft Foundry project
project = AIProjectClient(
    endpoint=os.getenv("AIPROJECT_ENDPOINT"),
    credential=DefaultAzureCredential()
)

#### Step 3: Connect to Azure AI Search

In [3]:
# First enter the name of your search index

index_name="health-plan"
print(index_name)

In [4]:
# Find Azure Cognitive Search connection
conn_id = None
for conn in project.connections.list():
    if getattr(conn, "type", None) == "CognitiveSearch":
        conn_id = conn.id
        break
if not conn_id:
    raise ValueError("No Azure Cognitive Search connection found in this project.")

# Define Azure AI Search tool and resources
ai_search_tool = AzureAISearchToolDefinition()
ai_search_resource = AzureAISearchToolResource(
    index_list=[
        AISearchIndexResource(
            index_connection_id=conn_id,
            index_name=index_name # Be sure to set your index name above
        )
    ]
)
tool_resources = ToolResources(azure_ai_search=ai_search_resource)

#### Step 4: Create the Search Agent
To create the Search Agent, we use the Azure AI Agent Service SDK to define a dedicated agent that specializes in searching our Azure AI Search index for health plan documents.

In [ ]:
# Search Agent
search_agent = project.agents.create_agent(
    model="gpt-4.1",
    name="search-agent",
    instructions="You are a helpful agent that is an expert at searching health plan documents.",
    tools=[ai_search_tool],
    tool_resources=tool_resources
)

#### Step 5: Create the Report Agent
Similarly, to create the Report Agent, we use the Azure AI Agent Service SDK to define an agent dedicated to generating detailed reports about health plans. This agent is configured with a specialized system prompt and can be easily orchestrated alongside other agents in the workflow.

In [ ]:
# Report Agent
report_agent = project.agents.create_agent(
    model="gpt-4.1",
    name="report-agent",
    instructions="You are a helpful agent that writes detailed reports about health plans."
)

#### Step 6: Create the Validation Agent
To create the Validation Agent, we again use the Azure AI Agent Service SDK to define an agent focused on validating that generated reports meet specific requirements. The Validation Agent is configured with instructions to check for required content (such as coverage exclusions) and to return a simple pass/fail result. This agent can be invoked programmatically as part of the multi-agent workflow, ensuring that all generated reports adhere to business rules before being delivered to the user.

In [ ]:
# Validation Agent
validation_agent = project.agents.create_agent(
    model="gpt-4.1",
    name="validation-agent",
    instructions="You are a helpful agent that validates reports. Return 'Pass' if the report meets requirements (must include coverage exclusions), otherwise return 'Fail'. Only return 'Pass' or 'Fail'."
)

### Part 2: Orchestrate the Multi-Agent System

Now that we've created our three task agents, the Search, Report, and Validation agents, we can put it all together and create a multi-agent system. We'll use Semantic Kernel to create an Orchestrator Agent that will leverage the three agents to create a report about a health plan.

When you run the below cell, you will see a chat box pop up at the top of VS Code asking you to input the name of a health plan. If you recall, we uploaded two health plans to the search index. Type one of the following health plans in the box and press enter to begin running the multi-agent system:

- Northwind Health Standard
- Northwind Health Plus

The orchestration code below...
- Defines an `orchestrate` function to coordinate the multi-agent workflow for a given health plan name:
  - The Search Agent retrieves information about the specified health plan from Azure AI Search.
  - The Report Agent generates a detailed report using the information returned by the Search Agent.
  - The Validation Agent checks that the report includes required content (coverage exclusions) and returns 'Pass' or 'Fail'.
  - If validation passes, the report is saved to a markdown file; otherwise, a message is printed indicating the report did not meet requirements.
- Defines a helper function to extract the last agent/assistant message from a list of messages.
- Provides a command-line interface to interactively enter health plan names, generate reports, and exit the system.
- Cleans up by deleting all agents when finished.

In [ ]:
def orchestrate(plan_name: str):
    # 1. Search Agent retrieves plan info
    search_thread_opts = AgentThreadCreationOptions(
        messages=[{"role": "user", "content": f"Tell me about the {plan_name} plan."}],
        tool_resources=tool_resources
    )
    search_run = project.agents.create_thread_and_process_run(
        agent_id=search_agent.id,
        thread=search_thread_opts
    )
    if search_run.status == "failed":
        raise RuntimeError(f"Search agent run failed: {search_run.last_error}")
    search_msgs = project.agents.messages.list(thread_id=search_run.thread_id)
    plan_info = extract_last_agent_message(search_msgs)

    # 2. Report Agent writes the report
    report_thread_opts = AgentThreadCreationOptions(
        messages=[{"role": "user", "content": f"Write a detailed report about the {plan_name} plan. Include coverage exclusions. Here is the relevant information: {plan_info}"}]
    )
    report_run = project.agents.create_thread_and_process_run(
        agent_id=report_agent.id,
        thread=report_thread_opts
    )
    if report_run.status == "failed":
        raise RuntimeError(f"Report agent run failed: {report_run.last_error}")
    report_msgs = project.agents.messages.list(thread_id=report_run.thread_id)
    report_content = extract_last_agent_message(report_msgs)

    # 3. Validation Agent checks the report
    validation_thread_opts = AgentThreadCreationOptions(
        messages=[{"role": "user", "content": f"Validate that the following report includes coverage exclusions. Here is the report: {report_content}"}]
    )
    validation_run = project.agents.create_thread_and_process_run(
        agent_id=validation_agent.id,
        thread=validation_thread_opts
    )
    if validation_run.status == "failed":
        raise RuntimeError(f"Validation agent run failed: {validation_run.last_error}")
    validation_msgs = project.agents.messages.list(thread_id=validation_run.thread_id)
    validation_result = extract_last_agent_message(validation_msgs)

    # 4. Output result
    if validation_result.strip().lower() == "pass":
        filename = f"{plan_name} Report.md"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(report_content)
        print(f"Report generated and saved to {filename}.")
        return {"report_was_generated": True, "content": report_content}
    else:
        print("Report did not meet validation requirements.")
        return {"report_was_generated": False, "content": "The report could not be generated as it did not meet the required validation standards."}

def extract_last_agent_message(messages):
    # Helper to extract the last agent/assistant message's text
    last_msg = None
    for msg in reversed(list(messages)):
        role = getattr(msg, "role", None)
        if role and ("agent" in role.lower() or "assistant" in role.lower()):
            last_msg = msg
            break
    if last_msg and getattr(last_msg, "content", None) and isinstance(last_msg.content, list):
        for part in last_msg.content:
            if part.get("type") == "text" and "text" in part and "value" in part["text"]:
                return part["text"]["value"]
    return ""

if __name__ == "__main__":
    print("Welcome to the Health Plan Multi-Agent System!")
    while True:
        plan_name = input("Enter the name of a health plan (or 'exit' to quit): ").strip()
        if not plan_name or plan_name.lower() == "exit":
            break
        result = orchestrate(plan_name)
        print(json.dumps(result, indent=2))

    # Cleanup: delete agents (optional, for resource management)
    project.agents.delete_agent(search_agent.id)
    project.agents.delete_agent(report_agent.id)
    project.agents.delete_agent(validation_agent.id)
    print("Agents deleted. Goodbye!")